In [13]:
import os

os.environ["OPENAI_API_BASE"] = "https://127.0.0.1:8080/v1"
os.environ["OPENAI_API_KEY"] = "your-api-key"


In [14]:
from langchain.prompts import PromptTemplate
# Use LangChain's OpenAI wrapper to connect to your local server
from langchain_community.chat_models import ChatOpenAI
import json
import re

In [15]:
llm = ChatOpenAI(
    openai_api_base="http://127.0.0.1:8080/v1",  # Local llama.cpp server
    openai_api_key="no-key-needed",  # Dummy key
    model="gemma-2-2b-it-Q8_0.gguf"
)

In [16]:
# Define the prompt template
template = """
Analyze this doctor-patient conversation and identify:
1. **Intents**: [schedule_test, referral, map_directions, none]
2. **Entities**: Test type, date, facility name, location, etc.

Return ONLY a JSON response with keys "intents" and "entities".

Examples:
Conversation: "You need a blood test next week."
Response: {{"intents": ["schedule_test"], "entities": {{"test_type": "blood test", "date": "next week"}}}}

Conversation: {text}
Response: 
"""

prompt = PromptTemplate(template=template, input_variables=["text"])


In [17]:
# Parse JSON output
def extract_json(response: str) -> dict:
    try:
        json_str = re.search(r'\{.*\}', response.content, re.DOTALL).group()
        return json.loads(json_str)
    except (AttributeError, json.JSONDecodeError):
        return {"intents": [], "entities": {}}

# Create the chain
chain = prompt | llm | extract_json

# Test with a sample conversation
conversation = "Refer the patient to Spine Center and schedule an X-ray."
result = chain.invoke({"text": conversation})

print(result)  # Output example: {"intents": ["referral", "schedule_test"], "entities": {"facility": "Spine Center", "test_type": "X-ray"}}

{'intents': ['referral', 'schedule_test'], 'entities': {'facility_name': 'Spine Center', 'test_type': 'X-ray'}}
